# Decision Trees
##### Sean Wade

In [1]:
from __future__ import division
import numpy as np
from collections import Counter
from scipy.stats import entropy
from mltools.dataManager import DataManager

from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('http://seanwade.com/jupyter.css').read())

In [11]:
data = DataManager('datasets/lenses.arff')

In [12]:
data.data.shape

(24, 4)

In [13]:
x_train, x_test, y_train, y_test = data.test_train_split()

In [14]:
def infoGain(data, labels, feature): 
    n, d = data.shape
    
    # Number of classes
    classes = np.unique(labels)
    
    # Values a feature can take
    unique_vals, counts = np.unique(data[:,feature], return_counts=True)
    
    total = 0
    for val, val_count in zip(unique_vals, counts):
        val_idx = np.where(data[:,feature] == val)[0]
        sub_data = data[val_idx]
        sub_label = labels[val_idx]
        
        probs = []
        class_cnts = Counter(sub_label)
        for cl in classes:
            probs.append(class_cnts[cl] / len(sub_label))
         
        total += (val_count / n) * entropy(probs)
    
    probs = []
    class_cnts = Counter(labels)
    for cl in classes:
        probs.append(class_cnts[cl] / len(labels))
        
    return entropy(probs) - total      

In [15]:
for i, x in enumerate(data.attr_names):
    print x, infoGain(data.data, data.labels, i)

age 0.0273075754262
spectacle-prescrip 0.0273868241754
astigmatism 0.261320112239
tear-prod-rate 0.380395665849
contact-lenses

IndexError: index 4 is out of bounds for axis 1 with size 4

In [16]:
def makeTree(data, labels, features, complete_data):
    num_data, num_features = data.shape
    unique_vals, counts = np.unique(labels, return_counts=True)
    
    if len(labels) == 0:
        return
    
    if len(unique_vals) == 1:
        return unique_vals[0]
    
    elif num_features == 0:
        return unique_vals[np.argmax(counts)]
    
    else:
        gain = np.zeros(num_features)
        for feature in range(num_features):
            gain[feature] = infoGain(data, labels, feature)
            
        best_feature = np.argmin(gain)
        feature_name = features[best_feature]
        tree = {feature_name: {}}
        unique_vals = np.unique(complete_data[:,best_feature])
        for i, value in enumerate(unique_vals):
            if counts[i] != 0:
                val_idx = np.where(data[:,best_feature] == value)[0]
                sub_data = data[val_idx]
                sub_label = labels[val_idx]

                sub_data = np.delete(sub_data, best_feature, axis=1)
                new_features = list(features)
                del new_features[best_feature]
                if len(val_idx) != 0:
                    sub_tree = makeTree(sub_data, sub_label, new_features, complete_data)
                    tree[feature_name][value] = sub_tree
                else:
                    tree[feature_name][value] = value
        complete_data = np.delete(complete_data, best_feature, axis=1)

            
        return tree

In [17]:
tree = makeTree(x_train, y_train, data.attr_names[:-1], np.vstack((x_train, x_test)))

IndexError: index 2 is out of bounds for axis 0 with size 2

In [18]:
# predict
n, m = x_test.shape
results = []
for i in range(n):
    data_point = x_test[i,:]
    features = data.attr_names
    sub_tree = tree
    while type(sub_tree) is dict :
        key = sub_tree.keys()[0]
        key_idx = features.index(key)
        sub_tree = sub_tree[key][data_point[key_idx]]
    results.append(sub_tree)

NameError: name 'tree' is not defined

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(y_test, results)

ValueError: Found input variables with inconsistent numbers of samples: [5, 0]

In [21]:
def makeTree(data, labels):
    default = labels[np.argmax(frequency)]
    if nData == 0 of nFeatures == 0:
        return default
    
    elif labels.count(classes[0]) == nData:
        # Only 1 class remains
        return classes[0] else:
        
    # Choose which feature is best
    gain = np.zeros(nFeatures)
    for feature in range(nFeatures):
        g = calc_info_gain(data,classes,feature)
        gain[feature] = totalEntropy - g 
    bestFeature = np.argmax(gain)
    tree = {featureNames[bestFeature]:{}} # Find the possible feature values
    for value in values:
    # Find the datapoints with each feature value
    for datapoint in data:
    if datapoint[bestFeature]==value:
    if bestFeature==0:
    datapoint = datapoint[1:] newNames = featureNames[1:]
    elif bestFeature==nFeatures: datapoint = datapoint[:-1] newNames = featureNames[:-1]
    else:
    datapoint = datapoint[:bestFeature] datapoint.extend(datapoint[bestFeature+1:]) newNames = featureNames[:bestFeature] newNames.extend(featureNames[bestFeature+1:])
    newData.append(datapoint)
    newClasses.append(classes[index]) index += 1
    # Now recurse to the next level
    subtree = make_tree(newData,newClasses,newNames) # And on returning, add the subtree on to the tree tree[featureNames[bestFeature]][value] = subtree
    return tree

SyntaxError: invalid syntax (<ipython-input-21-cda84b43424b>, line 3)

1.  (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits.  Otherwise, even with approaches like pruning below, the SS# type of overfit could still hurt us.) It is a good idea to use a simple data set (like the lenses data), which you can check by hand, to test your algorithm to make sure that it is working correctly. You should be able to get about 68% (61%-82%) predictive accuracy on lenses.

2. (15%) You will use your ID3 algorithm to induce decision trees for the cars data set and the voting data set.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  Note that with a full tree you will often get 100% accuracy on the training set. (Why would you and in what cases would you not?  This question is for our discussion, and should also be answered in your report.)  Note that you will need to support unknown attributes in the voting data set.  Use 10-fold CV on each data set to predict how well the models will do on novel data.  Report the training and test classification accuracy for each fold and then average the test accuracies to get your prediction.  Create a table summarizing these accuracy results, and discuss what you observed.  As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

3. (10%) For each of the two problems, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what rules it has discovered to try to solve each task).  If the tree is large you can just discuss a few of the more shallow attributes combinations and the most important decisions made high in the tree.

4.  (5%) How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

5.  (15%) Implement reduced error pruning to help avoid overfitting.  You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy.  Create a table comparing the original trees created with no overfit avoidance in item 2 above and the trees you create with pruning.  This table should compare a) the # of nodes (including leaf nodes) and tree depth of the final decision trees and b) the generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).

6.  (15%) Do an experiment of your own regarding decision trees.  You can be as creative as you would like on this.  Experiments could include such things as modifying the algorithm, modifying the measure of best attribute, comparing information gain and gain ratio, supporting real valued attributes, comparing different stopping criteria and/or pruning approaches, etc.  Be creative!